The second example notebook introduces a more complex use case, in that it deals with the simultaneous inference of the drift and diffusion functions. The necessary data is provided in the form of perturbed mean exit time values from the "Landau-Stuart process", which comprises a cubic drift and quadratic diffusion function. As of now, the computations are restricted to one-dimensional spacial domains.

The presented steps include data generation, linearized inference, MCMC sampling and the visualization of the results.

In [ ]:
import os
import numpy as np

from sp_inference import processes, model, sampling, logging

The configuration is centered around python dictionaries and numpy data structures. Most dictionaries serve as direct input for the library components, whereas the remaining dicts are just for concise representation of the settings.

In [ ]:
# Settings of the logger, which is shared by all library components
# The data is saved into a pre-defined file structure to reduce the number of settings
logSettings = {
    # Output folder; if None is specified, no data is saved
    "output_directory":     "02_example_stationary",
    # Decides if log info is printed to the screen
    "verbose":              True
}

# Settings for the generation of data from a prototypical stochastic process
dataSettings = {
    # Process type, check the 'processes' module for options
    "process_type":         "Dr31Di20Process",
    # Parameter(s) for the drift function (cubic and quadratic terms for Landau-Stuart process)
    "drift_parameters":     [2, 3],
    # Parameter(s) for the diffusion function (quadratic and linear for Landau-Stuart process)
    "diffusion_parameters": [1, 2],
    # Standard deviation of the zero-centered Gaussian noise on the generated data
    "standard_deviation":   0.01,
    # RNG seed for data point generation
    "rng_seed":             0,
    # Number of data points to generate
    "num_points":           50,
    # Spacial domain on which to compute data (here for the mean exit time)
    "domain_bounds":        [-1, 1]
}
# Settings for the inference model
modelSettings = {
    # Which parameter(s) to infer, can be 'drift', 'diffusion', or 'all'
    "params_to_infer":      "all",
    # Model type describing the PDE constraint of the inference problem and source of the data.
    # For options have a look at the implemented 'forms' in the 'pde_problems' module
    "model_type":           "mean_exit_time",
    # Determines if the underlying pde problem is treated as stationary or transient
    # All models can possibly be made transient, but for e.g. the mean exit time problem this would
    # lead to errornous results.
    "is_stationary":        True
}

# Settings of the Bi-Laplacian prior transferred from the hIPPYlib library
priorSettings = {
    # Mean function (for drift and diffusion)
    "mean_function":        [lambda x: -0.5*x, lambda x: 1.5*np.ones(x.size)],
    # Parameter for the covariance operator
    "gamma":                1,
    # Parameter for the covariance operator
    "delta":                1,
    # Determines if Robin boundary conditions are used for the computation of the covariance field
    "robin_bc":             False
}

# Settings for the FEM solver of the PDE constraint used for the inference problem
feSettings = {
    # Number of mesh points
    "num_mesh_points":      200,
    # Locations of the spacial domain boundaries
    "boundary_locations":   [-1, 1],
    # Dirichlet boundary values (homogeneous for mean exit time problem)
    "boundary_values":      [0, 0],
    # FEM element degrees for PDE solution (+ adjoint) and parameter function
    "element_degrees":      [1, 1]
}

# Settings of the solver for the linearized inference problem (MAP).
# This is an inexact Newton-CG solver with Armijo line search for globalization
solverSettings = {
    # Relative termination tolerance in the objective functional gradient norm
    "rel_tolerance":         1e-6,
    # Relative termination tolerance in the objective functional gradient norm
    "abs_tolerance":         1e-12,
    # Maximum number of iterations
    "max_iter":              50,
    # Number of Gauss Newton iterations before switching to Newton
    "GN_iter":               5,
    # Armijo constant for sufficient reduction
    "c_armijo":              1e-4,
    # Maximum number of backtracking iterations during line search
    "max_backtracking_iter": 10
}

# Settings for the construction of the reduced Hessian of the linearized problem about the MAP
hessianSettings = {
    # Number of generalized eigenvalue/-vector pairs to include
    "num_eigvals":           10,
    # Number of value to oversample for robustness of the randomized algorithm
    "num_oversampling":      10
}

# Settings for the setup of the MCMC sampler
samplerSettings = {
    # Algorithm type, available options are 'MALA' (gradient informed) and 'pCN' (random walk)
    "algorithm":             'MALA',
    # Determines if the MAP estimate is utilized for sampling, highly recommended
    "use_gr_posterior":      True,
    # Step size of the MALA sampler, this setting is called 'Beta' for the pCN sampler
    "StepSize":              0.1
}

# Settings for a sampler run
samplingRunSettings = {
    # Number of overall samples to produce, including burn-in
    "NumSamples":            1000,
    # Number of burn-in samples (discarded)
    "BurnIn":                100,
    # Variance for the random generation of the initial sample (if none is provided)
    "init_variance":         0.1,
    # RNG seed for the random generation of the initial sample (if none is provided)
    "init_seed":             0
}

# Postprocessing settings
visualizationSettings = {
    # Determines if plots are shown in the notebook
    "show":                  True,
    # Lag over which to evaluate the quantity of interest (QOI) from the MCMC run. If not defined
    # otherwise, the QOI is the Lebesgue norm of the parameter function over the defined domain.
    "qoi_lag":               100
}

In [ ]:
try:
    os.system('rm -r ' + logSettings["output_directory"])
except:
    pass

# The logger is instantiated as a separate object that is passed to other components
logger = logging.Logger(logSettings["verbose"],
                        logSettings["output_directory"])

We firstly create artificial data from a prototypical stochastic process. The spacial locations of the data points are passed to the generating routine for higher flexibility. The data is generated by superimposing zero-centered Gaussian noise to the exact value of the generating model, here the solution of the process's mean exit time problem. The resulting data is used for the definition of the misfit functional of the inference problem.


In [ ]:
randGenerator = np.random.default_rng(dataSettings["rng_seed"])
randLocs = randGenerator.uniform(*dataSettings["domain_bounds"], dataSettings["num_points"])
dataSettings["domain_points"] = randLocs

processType = processes.get_process(dataSettings["process_type"])
process = processType(dataSettings["drift_parameters"],
                      dataSettings["diffusion_parameters"],
                      logger)

# Generating function returns exact and perturbed data
forwardNoisy, forwardExact = process.generate_data(modelSettings["model_type"],
                                                   modelSettings["is_stationary"],
                                                   dataSettings)

exactDrift = process.compute_drift(randLocs)
exactDiffusion = process.compute_squared_diffusion(randLocs)
exactParamValues = np.column_stack((exactDrift, exactDiffusion))

exactParamData = [randLocs, exactParamValues]
randForwardData = [randLocs, forwardNoisy]
exactForwardData = [randLocs,  forwardExact]

# New settings dict for the misfit functional
misFitSettings = {
    "data_locations": randLocs,
    "data_values": forwardNoisy,
    "data_var": dataSettings["standard_deviation"]**2
}

The inference model simply takes settings for the overall model, its prior, the FEM setup and the misfit.

In [ ]:
inferenceModel = model.SDEInferenceModel(modelSettings,
                                         priorSettings,
                                         feSettings,
                                         misFitSettings,
                                         logger=logger)

# Get drift/diffusion mean and variance, along with mean exit time solution when using prior info
priorMeanData, priorVarianceData, priorForwardData = inferenceModel.get_prior_info("Randomized")

In [ ]:
test_data = priorMeanData[1]

An optimization routine computes the maximum a posteriori (MAP) estimate for the linearized problem. The algorithm employs a second order inexact Newton-CG algorithm to find the optimum. As a result, it returns the MAP point along with the reduced Hessian at that point (only considering a limited number of dominating eigenvalue-eigenvector pairs).

In [ ]:
# Data for mean, variance and forward solution is returned in form of x-y-value pairs over the spacial domain
mapMeanData, mapVarianceData, mapForwardData, hessEigVals \
    = inferenceModel.compute_gr_posterior(solverSettings, hessianSettings)

The MCMC sampler is created using the linearized models and the sampler specific settings. It is important that the linearized model has a well-defined posterior, that is has been 'run' successfully. The sampler can then create a sample collection and evaluate the associated QOI. Per default, the QOI is the L2-norm of the parameter function. Different QOIs can be defined as children of the 'BaseQOI' class in the sampling module and passed as an argument to the evaluation method.

In [ ]:
sampler = sampling.MCMCSampler(inferenceModel, samplerSettings, logger)
# Data for mean, variance and forward solution is returned in form of x-y-value pairs over the spacial domain
mcmcMeanData, mcmcVarianceData, mcmcForwardData = sampler.run(samplingRunSettings)
qoiTrace = sampler.evaluate_qoi()